# Data Mining Project
by Obsa Aba-waji Mirnary Carbajal

## Experiment 1: Linear Regression
In this notebook, we will be looking at a comparison of gas prices with fuel economy. Our hypothesis is that increases in gas prices are what drive the increase in improvements of fuel economy.

In [1]:
%matplotlib inline

## Importing Libs and Data

In [10]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from bokeh.io import output_notebook, show
from bokeh.plotting import figure, ColumnDataSource

from sklearn import linear_model
from sklearn.model_selection import train_test_split

In [4]:
vehicles = pd.read_csv('data/vehicles.csv', dtype=object)
weekly_gas_all_prices = pd.read_csv('data/GASALLCOVW.csv', index_col=0)
weekly_gas_reg_prices = pd.read_csv('data/GASREGCOVW.csv', index_col=0)
weekly_gas_prem_prices = pd.read_csv('data/GASPRMCOVW.csv', index_col=0)
annually_gas_all_prices = pd.read_csv('data/GASALLCOVA.csv', index_col=0)
annually_gas_reg_prices = pd.read_csv('data/GASREGCOVA.csv', index_col=0)
annually_gas_prem_prices = pd.read_csv('data/GASPRMCOVA.csv', index_col=0)

## Check to see that Vehicles Data has been loaded

Looking at the head of the data to make sure that the data has been loaded into their dataframes.

In [9]:
vehicles.head()
set(vehicles.fuelType.values)

{'CNG',
 'Diesel',
 'Electricity',
 'Gasoline or E85',
 'Gasoline or natural gas',
 'Gasoline or propane',
 'Midgrade',
 'Premium',
 'Premium Gas or Electricity',
 'Premium and Electricity',
 'Premium or E85',
 'Regular',
 'Regular Gas and Electricity',
 'Regular Gas or Electricity'}

## Cleaning up vehicles data

The vehicles data has a lot of fields that we will not be needing and is not ogranized in a logical manner for our needs.

In [11]:
columns_for_vehicles = ['year',
                        'make',
                        'model',
                        'VClass',
                        'fuelType',
                        'fuelType1',
                        'city08',
                        'highway08',
                        'comb08']
vehicles = vehicles[columns_for_vehicles]
vehicles.head()

,year,make,model,VClass,fuelType,fuelType1,city08,highway08,comb08
0,1985,Alfa Romeo,Spider Veloce 2000,Two Seaters,Regular,Regular Gasoline,19,25,21
1,1985,Ferrari,Testarossa,Two Seaters,Regular,Regular Gasoline,9,14,11
2,1985,Dodge,Charger,Subcompact Cars,Regular,Regular Gasoline,23,33,27
3,1985,Dodge,B150/B250 Wagon 2WD,Vans,Regular,Regular Gasoline,10,12,11
4,1993,Subaru,Legacy AWD Turbo,Compact Cars,Premium,Premium Gasoline,17,23,19


In [12]:
categorical_columns = ['make', 'VClass', 'fuelType', 'fuelType1']
numeric_columns = ['city08', 'highway08', 'comb08', 'year']

In [13]:
for col in vehicles.columns:
    if col in categorical_columns:
        vehicles[col] = pd.Categorical(vehicles[col])
        vehicles['{col}_codes'.format(col=col)] = vehicles[col].cat.codes
    elif col in numeric_columns:
        vehicles[col] = pd.to_numeric(vehicles[col])

In [14]:
vehicles.head()

,year,make,model,VClass,fuelType,fuelType1,city08,highway08,comb08,make_codes,VClass_codes,fuelType_codes,fuelType1_codes
0,1985,Alfa Romeo,Spider Veloce 2000,Two Seaters,Regular,Regular Gasoline,19,25,21,3,29,11,5
1,1985,Ferrari,Testarossa,Two Seaters,Regular,Regular Gasoline,9,14,11,38,29,11,5
2,1985,Dodge,Charger,Subcompact Cars,Regular,Regular Gasoline,23,33,27,31,28,11,5
3,1985,Dodge,B150/B250 Wagon 2WD,Vans,Regular,Regular Gasoline,10,12,11,31,30,11,5
4,1993,Subaru,Legacy AWD Turbo,Compact Cars,Premium,Premium Gasoline,17,23,19,117,0,7,4


## Cleaning up Gasoline data

The next piece of data we need to clean up is our gas prices data. Our vehicle data has years prior to 1993, but the data is not complete, so we will be focusing on gas prices starting in 1994.

In [15]:
annually_gas_prem_prices['prem_velocity'] = annually_gas_prem_prices.diff()
annually_gas_all_prices['all_velocity'] = annually_gas_all_prices.diff()
annually_gas_reg_prices['reg_velocity'] = annually_gas_reg_prices.diff()

In [16]:
temp = annually_gas_all_prices.join(annually_gas_prem_prices)
joined_annually_gas_prices = temp.join(annually_gas_reg_prices)
len(joined_annually_gas_prices)

22

In [17]:
joined_annually_gas_prices.head()

,GASALLCOVA,all_velocity,GASPRMCOVA,prem_velocity,GASREGCOVA,reg_velocity
DATE,,,,,,
1995-01-01,1.144,NaN,1.287,NaN,1.103,0.031
1996-01-01,1.231,0.087,1.371,0.084,1.192,0.089
1997-01-01,1.228,-0.003,1.368,-0.003,1.189,-0.003
1998-01-01,1.056,-0.172,1.199,-0.169,1.017,-0.172
1999-01-01,1.156,0.100,1.297,0.098,1.116,0.099


In [18]:
joined_annually_gas_prices.index = joined_annually_gas_prices.index.astype(str).str[:4].astype(int)
joined_annually_gas_prices.index.names = ['year']
joined_annually_gas_prices.head()

,GASALLCOVA,all_velocity,GASPRMCOVA,prem_velocity,GASREGCOVA,reg_velocity
year,,,,,,
1995,1.144,NaN,1.287,NaN,1.103,0.031
1996,1.231,0.087,1.371,0.084,1.192,0.089
1997,1.228,-0.003,1.368,-0.003,1.189,-0.003
1998,1.056,-0.172,1.199,-0.169,1.017,-0.172
1999,1.156,0.100,1.297,0.098,1.116,0.099


## Joining Vehicle Data and Gasoline Data
Now we will be joining the two tables together by year. 

In [19]:
data = vehicles.join(joined_annually_gas_prices, how='outer', on='year')
data = data.dropna(axis=0, how='any')
data.head()

,year,make,model,VClass,fuelType,fuelType1,city08,highway08,comb08,make_codes,VClass_codes,fuelType_codes,fuelType1_codes,GASALLCOVA,all_velocity,GASPRMCOVA,prem_velocity,GASREGCOVA,reg_velocity
2788,1996,Acura,NSX,Two Seaters,Premium,Premium Gasoline,16,22,18,2,29,7,4,1.231,0.087,1.371,0.084,1.192,0.089
2789,1996,Acura,NSX,Two Seaters,Premium,Premium Gasoline,16,22,18,2,29,7,4,1.231,0.087,1.371,0.084,1.192,0.089
2790,1996,BMW,Z3 Roadster,Two Seaters,Premium,Premium Gasoline,20,29,23,11,29,7,4,1.231,0.087,1.371,0.084,1.192,0.089
2792,1996,BMW,Z3 Roadster,Two Seaters,Premium,Premium Gasoline,20,29,23,11,29,7,4,1.231,0.087,1.371,0.084,1.192,0.089
2793,1996,Chevrolet,Corvette,Two Seaters,Regular,Regular Gasoline,15,23,18,24,29,11,5,1.231,0.087,1.371,0.084,1.192,0.089


## Experiment

Now that the data has been cleaned, we can focus on the first experiment. In this experiment, we will be looking at the speed changes of gas prices over the years, then comparing their effect to the effect of increases in fuel effiency. 

### Splitting the data
The first step is going to be splitting out data into testing and training data as well as into input data vs output data

In [40]:
x = data.VClass_codes.values
y = data.comb08.values
x = x.reshape(len(x), 1)
y = y.reshape(len(y), 1)

[29 29 29 ...,  0 27 15]
[18 18 23 ..., 30 86 16]


In [45]:
x = data.make_codes.values
y = data.comb08.values
x = x.reshape(len(x), 1)
y = y.reshape(len(y), 1)

In [47]:
x = data.fuelType_codes.values
y = data.comb08.values
x = x.reshape(len(x), 1)
y = y.reshape(len(y), 1)

In [49]:
x = data.all_velocity.values
y = data.comb08.values
x = x.reshape(len(x), 1)
y = y.reshape(len(y), 1)

In [30]:
X_train, X_test, y_train, y_test = train_test_split(data[['make_codes', 'year']], data[['comb08']])
print(X_train.shape)
print(y_train.shape)

(16618, 2)
(16618, 1)


In [32]:
regr = linear_model.LinearRegression()
regr.fit(X_train, y_train)
regr.pr

# plot it as in the example at http://scikit-learn.org/
# Error with matplotlib not SKLearn.
plt.scatter(X_train, y_train,  color='black')
plt.plot(X_test, regr.predict(X_test), color='blue', linewidth=3)
plt.xticks(())
plt.yticks(())
plt.show()

AttributeError: 'LinearRegression' object has no attribute 'pr'